# MedNER
---  
  
Medikuntzako NER sistema gainbegiratu bat egingo da.

Horretarako, lehenik eta behin beharrezko liburutegiak inportatuko ditugu. Erabiliko dugu the Hugging Face Transformers liburutegia zeinak diseinatuta dagoen aurre-entrenatutako Transformer modeloak, BERT adibidez, erabiltzeko. Instalatuko dugu:


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 8.6 MB/s 
     |████████████████████████████████| 182 kB 48.4 MB/s 
     |████████████████████████████████| 7.6 MB 8.0 MB/s 


Gainerako liburutegiak inportatuko ditugu:

In [7]:
from transformers import BertModel, BertTokenizer, BertTokenizerFast
import torch
from tqdm import tqdm
import torch.nn as nn
import numpy as np
import random
import time
import spacy
import os

Konprobatuko dugu notebook-a GPUan exekutatzen dagoela ("Running on cuda" inprimatu beharko luke).

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on {}".format(device))

Running on cpu


Datuak dituen direktoria erabiliko dugu:

In [11]:
from google.colab import drive
drive.mount('/content/drive')

%cd "/content/drive/MyDrive/universidad/4.CURSO/HP/PROIEKTUA/data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/universidad/4.CURSO/HP/PROIEKTUA/data


## Datuak irakurri eta *train*, *dev* eta *test* partizioak egin
Gure anotatutako corpus guztia *corpus_pubtator.txt* fitxategian dago.  Datu hauek PubTator formatuan daude. Zehazki kasu bakoitza honela dago errepresentatuta:
  
```
PMID | t | Title text  
PMID | a | Abstract text    
PMID TAB StartIndex TAB EndIndex TAB MentionTextSegment TAB SemanticTypeID TAB EntityID
...
```

Lehenengo bi lerroek izenburuko eta laburpeneko testuak aurkezten dituzte (lerro-jauzirik eta tabulaziorik gabe testuan). Hurrengo lerroek aipamenak aurkezten dituzte, lerro bakoitzeko bana. *StartIndex* eta *EndIndex* dokumentuaren testuan 0an oinarritutako karaktereen aurkibideak dira, Izenburua eta Laburpena kateatuz eraikiak, SPACE karaktere batez bereiziak. *MentionTextSegment* karaktere-posizio horien arteko benetako aipamena da. *EntityID* UMLS entitatearen id da (kontzeptua), eta *SemanticTypeID* entitatea UMLSen lotuta dagoen mota semantikoaren id da. UMLS entitate mota semantiko bati baino gehiagori lotuta badago, eremu honek komen arabera bereizitako zerrenda bat du ID mota guztiekin. 2017-AA bertsio aktiboan ez dauden UMLS kontzeptu guztiak *UnknownType* mota semantiko bereziari lotuta daude.  

Jarraian adibide bat azaltzen da:
```
25763772|t|DCTN4 as a modifier of chronic Pseudomonas aeruginosa infection in cystic fibrosis
25763772|a|Pseudomonas aeruginosa (Pa) infection in cystic fibrosis (CF) patients is associated with worse long-term pulmonary disease and shorter survival, and chronic Pa infection (CPA) is associated with reduced lung function, faster rate of lung decline, increased rates of exacerbations and shorter survival. By using exome sequencing and extreme phenotype design, it was recently shown that isoforms of dynactin 4 (DCTN4) may influence Pa infection in CF, leading to worse respiratory disease. The purpose of this study was to investigate the role of DCTN4 missense variants on Pa infection incidence, age at first Pa infection and chronic Pa infection incidence in a cohort of adult CF patients from a single centre. Polymerase chain reaction and direct sequencing were used to screen DNA samples for DCTN4 variants. A total of 121 adult CF patients from the Cochin Hospital CF centre have been included, all of them carrying two CFTR defects: 103 developed at least 1 pulmonary infection with Pa, and 68 patients of them had CPA. DCTN4 variants were identified in 24% (29/121) CF patients with Pa infection and in only 17% (3/18) CF patients with no Pa infection. Of the patients with CPA, 29% (20/68) had DCTN4 missense variants vs 23% (8/35) in patients without CPA. Interestingly, p.Tyr263Cys tend to be more frequently observed in CF patients with CPA than in patients without CPA (4/68 vs 0/35), and DCTN4 missense variants tend to be more frequent in male CF patients with CPA bearing two class II mutations than in male CF patients without CPA bearing two class II mutations (P = 0.06). Our observations reinforce that DCTN4 missense variants, especially p.Tyr263Cys, may be involved in the pathogenesis of CPA in male CF.
25763772        0       5       DCTN4   T116,T123    C4308010
25763772        23      63      chronic Pseudomonas aeruginosa infection        T047    C0854135
25763772        67      82      cystic fibrosis T047    C0010674
25763772        83      120     Pseudomonas aeruginosa (Pa) infection   T047    C0854135
...
```

Bestalde, *train*, *dev* eta *test* partizioak *corpus_pubtator_pmids_trng.txt, corpus_pubtator_pmids_dev.txt, corpus_pubtator_pmids_test.txt* fitxategietan daude.  Hauek dokumentuaren  %60, %20, %20-ko ausazko partizioak dituzte, hurrenez hurren. Bakoitzean agertzen dira PMID kodeak zehaztuz bakoitzean dauden kasue kodeak.  

Beraz, partizioak egingo ditugu dokumentu bana izateko *train*, *dev* eta *test* kasuetarako. Horretarako:
1. Sortuko dugu fitxategi bana *train*, *dev* eta *test*-eko kasuak gordetzeko.
2. Irakurriko ditugu *corpus_pubtator_pmids_trng.txt, corpus_pubtator_pmids_dev.txt, corpus_pubtator_pmids_test.txt* fitxategiak. Irakurtzen dugun kode bakoitzeko *corpus_pubtator.txt* fitxategian  dagoen kasu hori hartuko dugu eta dagokion fitxategian (train*, *dev* edo *test*) kopiatuko dugu.

### Aldagai orokorrak definitu direktorioak gordetzeko

In [12]:
train_directory = "corpus_pubtator_pmids_trng.txt"
dev_directory = "corpus_pubtator_pmids_dev.txt"
test_directory = "corpus_pubtator_pmids_test.txt"

train_def_directory = "corpus_pubtator_train.txt"
dev_def_directory = "corpus_pubtator_dev.txt"
test_def_directory = "corpus_pubtator_test.txt"

### Fitxategiak sortu

In [16]:
def createFile(path):
    if not os.path.exists(path):
        os.makedirs(os.path.dirname(path), exist_ok=True)

def createDirectory(path):
    if not os.path.exists(path):
        os.mkdir(path)

createFile("./" + train_def_directory)
createFile("./" + dev_def_directory)
createFile("./" + test_def_directory)

### Fitxategiak irakurri

In [ ]:
def read_txt(filename):
    
    """ Read input one by line """
    
    sentences=[]
    sentence=[]
    with open(filename) as file:
        for line in file:
            cols=line.rstrip().split("\t")
            if len(cols) < 2:
                if len(sentence) > 0:
                    sentences.append(sentence)
                sentence=[]
                continue
                
            word=cols[0]
            tag=cols[1]
            
            sentence.append((word, tag))
            
        if len(sentence) > 0:
            sentences.append(sentence)
            
    return sentences

## Z1
MedMentions corpusa erabiliz NER sistema orokor bat
entrenatu, termino bat UMLSkoa den, hau da, osasunarekin zerikusia ote duen
jakiteko. Hau da, terminoak identifikatu, klase bakarra irteeran duzularik
(Medikuntzkoa edo MED).  

[DCTN4] as a modifier of [chronic Pseudomonas aeruginosa infection] in
[cystic fibrosis]